<a href="https://colab.research.google.com/github/susichao/ml-project/blob/main/DGA_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pushbullet.py
!pip install tldextract
!pip install scapy
!pip install configparser


Import necessary libraries and modules

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import tldextract
import scipy
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

creating a Settings.conf file

In [ ]:
import configparser

config = configparser.ConfigParser()
config['Settings'] = {'baseline': '0.5', 'total_bigrams': '1000'}

with open('settings.conf', 'w') as configfile:
    config.write(configfile)


Load training data and settings

In [ ]:

import os
import pandas as pd

def load_data_and_settings() -> tuple:
    if os.path.isfile('data/settings.conf'):
        data = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with your CSV file path
        baseline, total_bigrams_settings = load_settings()  # Assuming load_settings() loads settings from 'settings.conf'
        return data, baseline, total_bigrams_settings
    else:
        raise ValueError("Error loading data. Please check the files.")

In [ ]:
# import configparser

# def load_settings():
#     config = configparser.ConfigParser()
#     config.read_string(uploaded['settings.conf'].decode('utf-8'))

#     baseline = float(config['Settings']['baseline'])
#     total_bigrams_settings = int(config['Settings']['total_bigrams'])

#     return baseline, total_bigrams_settings

# baseline, total_bigrams_settings = load_settings()
# print("Baseline:", baseline)
# print("Total Bigrams Settings:", total_bigrams_settings)


entropy of a domain name

In [ ]:
# import math
# from collections import Counter

# def calculate_entropy(domain: str) -> float:
#     char_freq = Counter(domain)

#     domain_length = len(domain)
#     prob_dist = [char_freq[char] / domain_length for char in set(domain)]

#     entropy_val = -sum(p * math.log2(p) for p in prob_dist)

#     return entropy_val

# def calculate_entropy_for_dataframe(data: pd.DataFrame, domain_column: str) -> pd.Series:
#     entropies = data[domain_column].apply(calculate_entropy)
#     return entropies

# data = pd.read_csv('your_data.csv')  # Load your CSV dataset
# entropies = calculate_entropy_for_dataframe(data, 'domain')

# # Add entropy values to the DataFrame
# data['entropy'] = entropies

# # Print the DataFrame with entropy values
# print(data)

lexical features from domain names

In [ ]:

    # Implement lexical feature extraction logic

def extract_lexical_features(domain_list):
    features = []

    for domain in domain_list:
        # Length of the domain
        length = len(domain)

        # Count of numbers, letters, special characters
        num_count = sum(char.isdigit() for char in domain)
        letter_count = sum(char.isalpha() for char in domain)
        special_char_count = length - num_count - letter_count

        # Number of vowels and consonants
        vowels = sum(char in 'aeiou' for char in domain.lower())
        consonants = letter_count - vowels

        # Count of consecutive characters
        consecutive_count = max(sum(1 for _ in g) for _, g in groupby(domain.lower()))

        # Ratio of vowels to consonants
        if consonants != 0:
            vowel_consonant_ratio = vowels / consonants
        else:
            vowel_consonant_ratio = np.nan

        # Ratio of numbers to letters
        if letter_count != 0:
            number_letter_ratio = num_count / letter_count
        else:
            number_letter_ratio = np.nan

        # Add features to the feature vector
        feature_vector = [length, num_count, letter_count, special_char_count,
                          vowels, consonants, consecutive_count, vowel_consonant_ratio,
                          number_letter_ratio]

        features.append(feature_vector)

    return features


clustering features from domain names

In [ ]:
def extract_clustering_features(domain: str) -> list:
    # Implement clustering feature extraction logic here
    # For example, use K-Means clustering
    kmeans = KMeans(n_clusters=5)
    features = kmeans.fit_transform([domain])
    return features

 extract bigram features from domain names

In [ ]:
def extract_bigram_features(domain: str) -> list:
    features = []
    for i in range(len(domain) - 1):
        features.append(domain[i:i+2])
    return features

extracting features from domain name


In [ ]:
def extract_features(domain: str) -> list:
    features = []
    features.extend(extract_bigram_features(domain))
    features.append(calculate_entropy(domain))
    features.extend(extract_lexical_features(domain))
    features.extend(extract_clustering_features(domain))
    return features

train the ML model

In [ ]:
def train_model():
    bigram_dict, baseline, total_bigrams_settings = load_data_and_settings()
    if bigram_dict:
        X, y = [], []
        data = pd.read_csv('data/training_data.txt')
        for domain in data:
            features = extract_features(domain)
            X.append(features)
            label = 1 if check_domain(domain) == 1 else 0
            y.append(label)

vectorize features

In [ ]:
 vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
        X = vectorizer.fit_transform(X)

split data into training and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Training and Evaluation

In [ ]:
 # Train the ML model
        model = RandomForestClassifier()
        model.fit(X_train, y_train)

In [ ]:
 # Evaluate the model
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print("Model accuracy:", accuracy)
    else:
        raise ValueError("Error loading data. Model training failed.")


In [ ]:
# Main function to execute the ML model training
if __name__ == "__main__":
    train_model()